In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import bigrams, FreqDist
import re
import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer, AutoModel
from keras.preprocessing.text import Tokenizer


ARABIC_LETTERS_LIST = [
    'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط',
    'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي'
]

DIACRITICS_LIST = [
    'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ'
]

ALL_CHARACTERS = ARABIC_LETTERS_LIST + DIACRITICS_LIST


CHAR_TO_INDEX = {char: index for index, char in enumerate(ALL_CHARACTERS)}

INDEX_TO_CHAR = {index: char for index, char in enumerate(ALL_CHARACTERS)}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Seif\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Seif\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%tensorflow_version 1.x
import time
import random
import numpy as np
import pickle as pkl
import tensorflow as tf

from keras.models import Model, Input
from keras.layers import Embedding, Dense, Dropout, LSTM, CuDNNLSTM, Bidirectional, TimeDistributed
from keras.utils import Sequence
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.initializers import glorot_normal
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.layers import Activation
from keras.losses import sparse_categorical_crossentropy
from keras.layers import LSTM


UsageError: Line magic function `%tensorflow_version` not found.


In [3]:

def cleaningEnglishAndNumbers(text): 
    text = re.sub(r"[a-zA-Z0-9٠-٩]", "", text)
    return text


def cleaningSpaces(text):
    text = re.sub(r"[\s]+", " ", text)
    return text


def Remove(text):
    text = re.sub(r"[[]]", "", text)
    text = re.sub(r"[]]", "", text)
    text = re.sub(r"[[]", "", text)
    text = re.sub(r"-", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"[،؛؟():.]", "", text)
    text = re.sub(r"[{}}!*;»«]", "", text)
    return text
    

def RemoveSingleLetters(text):
    #remove single letters [أ-ي] boundered with spaces
    text = re.sub(r"\s[أ-ي]\s", "", text)
    return text




In [4]:
file_path = 'dataset/val.txt'

def read_file_content(file_path, encoding='utf-8'):
    with open(file_path, 'r', encoding=encoding) as file:
        return file.read()

file_content = read_file_content(file_path)

#cleaning text

file_content = cleaningEnglishAndNumbers(file_content)
file_content = cleaningSpaces(file_content)
file_content = Remove(file_content)
file_content = RemoveSingleLetters(file_content)


# add it in txt file
with open('dataset/Updatedtrain.txt', 'w', encoding='utf-8') as file: 
    file.write(file_content)

print('Train raws:', len(file_content))

Train raws: 910274


C:\Users\Seif\AppData\Local\Temp\ipykernel_11868\1934813480.py:12: FutureWarning: Possible nested set at position 1
  text = re.sub(r"[[]]", "", text)
C:\Users\Seif\AppData\Local\Temp\ipykernel_11868\1934813480.py:14: FutureWarning: Possible nested set at position 1
  text = re.sub(r"[[]", "", text)


In [5]:

def remove_diactrics(text):
    text = re.sub(r"[\u064B-\u065F]", "", text) 
    return text

file_content_without_diactrics = remove_diactrics(file_content)

with open('dataset/UpdatedtrainWithoutDiactrics.txt', 'w', encoding='utf-8') as file: 
    file.write(file_content_without_diactrics)


print('Train raws without diactrics:', len(file_content_without_diactrics))

Train raws without diactrics: 542752


In [6]:

def tokenize(text):
    return [text[i:i+500] for i in range(0, len(text), 500)]

file_content_tokenized = tokenize(file_content_without_diactrics)


In [7]:
#using bag of words
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(file_content_tokenized)

with open('dataset/BoW_representation.txt', 'w', encoding='utf-8') as file:
    for row in X.toarray():
        file.write(" ".join(map(str, row)) + "\n")

print('Train rows:', len(file_content))
print('Train rows without diacritics:', len(file_content_without_diactrics))

Train rows: 910274
Train rows without diacritics: 542752


In [8]:
#using BERT embeddings

model_name = "bert-base-multilingual-cased"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
encoded_texts = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in file_content_tokenized]

with open('dataset/BERT_embeddings.txt', 'w', encoding='utf-8') as file:
    for embeddings in encoded_texts:
        file.write(" ".join(map(str, embeddings)) + "\n")

print('Train rows:', len(file_content))

Train rows: 910274


In [9]:
#one hot encoding
